In [1]:
!pip install langchain_huggingface
!pip install langchain
!pip install langchain-community
!pip install serpapi
!pip install google-search-results


In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper
from google.colab import userdata
import os

SERPAPI_API_KEY = userdata.get("SERPAPI_API_KEY")
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

HUGGINGFACEHUB_API_KEY = userdata.get("HUGGINGFACEHUB_API_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [19]:
!git config --global credential.helper cache
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).

In [20]:
llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")
chat_model=ChatHuggingFace(llm=llm)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
# setup tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [22]:
# setup ReAct style prompt
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=', '.join([t.name for t in tools])
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [26]:
chat_model_with_stop = chat_model.bind(stop=["\nObservation"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

In [28]:
# instantiate AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

agent_executor.invoke(
    {
        "input": "Who is the current holder of the speed skating world record on 500 meters? What is her current age raised to the 0.43 power?"
    }
)

Thought: I think I can answer this question using the Search and Calculator tools provided.

Action:

```json
{
  "action": "Search",
  "action_input": "current holder of speed skating world record 500 meters"
}
```

Observation: Iris Tussenraad from the Netherlands holds the current world record for the 500m distance in women's speed skating{'type': 'organic_result', 'title': 'Current world records Men', 'source': 'SpeedSkatingStats.com'}Could not parse LLM output: I'll use the Calculator tool to find Iris Tussenraad's age raised to the power of 0.43.

Action:

```json
{
  "action": "Calculator",
  "args": [Iris Tussenraad, "age", 2021],
  "callbacks": [MyCallbackHandler()],
  "tags": ["age"],
  "metadata": {"Invalid or incomplete responseCould not parse LLM output: Thought: Hmm, it seems the LLM output is not in a suitable format for the Calculator tool. I'll try to find Iris Tussenraad's age online and then use the Calculator tool to raise it to the power of 0.43.

Action:

```json


{'input': 'Who is the current holder of the speed skating world record on 500 meters? What is her current age raised to the 0.43 power?',
 'output': 'The current holder of the speed skating world record on 500 meters is Iris Tussenraad from the Netherlands, and as'}